# Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import cv2
import pandas as pd
from patchify import patchify,unpatchify
import tifffile as tif
from tqdm import tqdm
import gc
import utils

c:\Users\Administrator\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Helping Functions

In [4]:
# create data frame to handle all ROI images

df =pd.DataFrame()
pred_dir = 'predicted_images'
img_gt_dir = 'test_images'
img_paths =[]
gt_paths =[]
pred_paths =[]
imgs_list = []
models_name = []

for model_pred_dir in sorted(os.listdir(pred_dir)):
    for pred_file in [x for x in sorted(os.listdir(pred_dir+'/'+model_pred_dir)) if x!='desktop.ini']:
        pred_paths.append(pred_dir+'/'+model_pred_dir+'/'+pred_file)
        imgs_list.append(pred_file[:-4])
        models_name.append(model_pred_dir)

df['img_name'] = imgs_list
df['img_path'] =''
df['gt_path'] =''
df['pred_path'] = pred_paths
df['models_name'] = models_name
for img_file in set(imgs_list):
    [im_f , gt_f] = sorted(os.listdir(f'{img_gt_dir}/{img_file}'))[:-1]
    for idx, row in  df[df['img_name']==img_file].iterrows():
        df.at[idx,'img_path']= f'{img_gt_dir}/{img_file}/{im_f}'
        df.at[idx, 'gt_path'] = f'{img_gt_dir}/{img_file}/{gt_f}'

df

,img_name,img_path,gt_path,pred_path,models_name
0,102,test_images/102/102.tif,test_images/102/102_mask.tiff,predicted_images/VGG_unet_90_20_model/102.png,VGG_unet_90_20_model
1,121,test_images/121/121.tif,test_images/121/121_mask.tiff,predicted_images/VGG_unet_90_20_model/121.png,VGG_unet_90_20_model
2,201,test_images/201/201.tif,test_images/201/201_mask.tiff,predicted_images/VGG_unet_90_20_model/201.png,VGG_unet_90_20_model
3,202,test_images/202/202.tif,test_images/202/202_mask.tiff,predicted_images/VGG_unet_90_20_model/202.png,VGG_unet_90_20_model
4,203,test_images/203/203.tif,test_images/203/203_mask.tiff,predicted_images/VGG_unet_90_20_model/203.png,VGG_unet_90_20_model
...,...,...,...,...,...
145,395,test_images/395/395.tiff,test_images/395/395_mask.tiff,predicted_images/unet_98_3_model/395.png,unet_98_3_model
146,399,test_images/399/399.tiff,test_images/399/399_mask.tiff,predicted_images/unet_98_3_model/399.png,unet_98_3_model
147,401,test_images/401/401.tif,test_images/401/401_mask.tiff,predicted_images/unet_98_3_model/401.png,unet_98_3_model
148,402,test_images/402/402.tif,test_images/402/402_mask.tiff,predicted_images/unet_98_3_model/402.png,unet_98_3_model


In [34]:
# iterate over the images to do:
# 1- border to each images
# 2- process the mask image
# 3- merge mask with main image
# 4- merge predicted mask with main image

# create dataFrame to save the testing results

columns = ['img_name', 'models_name', 'iou_before', 'iou_after']

df_result = pd.DataFrame([], columns=columns)
df_result

result_after_proc_dir = 'whole_slide_mask_before_after/'

for idx, row in df[~(df['models_name'].isin(['VGG_unet_90_20_model', 'unet_100_0_model', 'res_unet_100_0_model']))].iterrows():
    print(f'Model {df.iloc[idx]["models_name"]}.............\n')
    # create folders for each model
    
    if not os.path.exists(result_after_proc_dir):
        os.mkdir(result_after_proc_dir)

    if not os.path.exists(f'{result_after_proc_dir}{df.iloc[idx]["models_name"]}'):
        os.mkdir(f'{result_after_proc_dir}{df.iloc[idx]["models_name"]}')

    # create folder for each ROI
    if not os.path.exists(f'{result_after_proc_dir}{df.iloc[idx]["models_name"]}/{df.iloc[idx]["img_name"]}'):
        os.mkdir(f'{result_after_proc_dir}{df.iloc[idx]["models_name"]}/{df.iloc[idx]["img_name"]}')

    # get processed mask

    iou_before, result_before, iou_after, result_after, processed_msk =  utils.calculate_iou_processed_mask(df.iloc[idx])
    # processed_msk = utils.get_processed_predMask(df.iloc[idx]) 
    data_row = []
    data_row.append(row['img_name'])
    data_row.append(row['models_name'])
    data_row.append(iou_before)
    data_row.append(iou_after)
    df_result.loc[len(df_result)] = data_row

    path = f'{result_after_proc_dir}{data_row[1]}/{data_row[0]}'

    cv2.imwrite(f'{path}/{data_row[0]}_processed_mask.png',processed_msk) # save the processed mask

    cv2.imwrite(f'{path}/{data_row[0]}_iou_before.png',result_before) # save the processed mask
    cv2.imwrite(f'{path}/{data_row[0]}_iou_after.png',result_after) # save the processed mask
    # merge image with processed mask
    # img = utils.get_patchifiable(cv2.imread(df.iloc[idx]['img_path']))


    # pred_pre = utils.get_patchifiable(cv2.imread(df.iloc[idx][3],0))
    # merged_before_processing = utils.get_merged_img(img, pred_pre)
    # cv2.imwrite(f'{path}/{data_row[0]}_merged_before_processing.tiff',merged_before_processing)

    
    # merged_after_processing = utils.get_merged_img(img, processed_msk)
    # cv2.imwrite(f'{path}/{data_row[0]}_merged_after_processing.tiff',merged_after_processing) # save the processed mask




Model unet_100_50_model.............

IOU : 0.22977762818401654
search for intersect in  218 contours


100%|██████████| 218/218 [03:44<00:00,  1.03s/it]


(6144, 25600) (6144, 25600)
IOU : 0.2388482031481284
Model unet_100_50_model.............

IOU : 0.18182292507862652
search for intersect in  128 contours


100%|██████████| 128/128 [01:28<00:00,  1.44it/s]


(16896, 6144) (16896, 6144)
IOU : 0.14452807337755455
Model unet_100_50_model.............

IOU : 0.16926776961252293
search for intersect in  148 contours


...

(1024, 512) (1024, 512)
IOU : 0.8763609171926182
Model unet_98_3_model.............

IOU : 0.09983378934394488
search for intersect in  18 contours


100%|██████████| 18/18 [00:01<00:00, 11.48it/s]


(3584, 3584) (3584, 3584)
IOU : 0.1771197535213119
Model unet_98_3_model.............

IOU : 0.876888792771107
search for intersect in  1 contours


100%|██████████| 1/1 [00:00<00:00, 399.42it/s]

(512, 512) (512, 512)
IOU : 0.877599781764136


In [37]:
df_result

,img_name,models_name,iou_before,iou_after
0,102,VGG_unet_90_20_model,0.308638,0.310138
1,121,VGG_unet_90_20_model,0.772153,0.818498
2,201,VGG_unet_90_20_model,0.875295,0.897302
3,202,VGG_unet_90_20_model,0.813242,0.634915
4,203,VGG_unet_90_20_model,0.802578,0.887642
...,...,...,...,...
145,395,unet_98_3_model,0.096313,0.445421
146,399,unet_98_3_model,0.205705,0.609675
147,401,unet_98_3_model,0.872441,0.876361
148,402,unet_98_3_model,0.099834,0.177120


In [36]:
df_result.to_csv('all_models_results.csv', index=False)